In [33]:
import pandas as pd
url='https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
dat=pd.read_csv(url)

In [2]:
dat.head()

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [3]:
columns=['Age','working_agency','empid','education','educationno','marital-status','occupation','relationship','race','sex','capital-gain','capital_loss','hours_per_week','native_country','salary']
data=pd.read_csv(url,names=columns)

In [4]:
data.head()

,Age,working_agency,empid,education,educationno,marital-status,occupation,relationship,race,sex,capital-gain,capital_loss,hours_per_week,native_country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
data['working_agency'].unique()

array([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
       ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay',
       ' Never-worked'], dtype=object)

In [6]:
import numpy as np
data.replace(' ?',value=np.nan,inplace=True)


In [7]:
data['working_agency'].unique()

array([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
       ' Local-gov', nan, ' Self-emp-inc', ' Without-pay',
       ' Never-worked'], dtype=object)

In [8]:
data.dropna(inplace=True)

In [9]:
data['working_agency'].isna().sum()
0

0

In [10]:
data['salary'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [11]:
x=data.drop(['salary'],axis=1)
y=data['salary'].replace([' <=50K', ' >50K'],value=[0,1])

In [12]:
X=pd.get_dummies(x)

In [13]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)



In [14]:
from sklearn.model_selection import train_test_split,GridSearchCV
train_x,test_x,train_y,test_y=train_test_split(X_scaled,y,test_size=0.3,random_state=33)


In [15]:
from sklearn.linear_model import LogisticRegression
logisticmodel=LogisticRegression()

In [16]:
para={
   'penalty':['l1', 'l2'],
}

In [22]:
grid=GridSearchCV(logisticmodel,param_grid=para)

In [23]:
grid.fit(train_x,train_y)

/home/saibeast/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/saibeast/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/saibeast/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/saibeast/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"


GridSearchCV(estimator=LogisticRegression(),
             param_grid={'penalty': ['l1', 'l2']})

In [24]:
grid.best_params_

{'penalty': 'l2'}

In [16]:
logisticmodel=LogisticRegression(penalty='l2',random_state=33)

In [17]:
logisticmodel.fit(train_x,train_y)

LogisticRegression(random_state=33)

In [18]:
logisticmodel.score(test_x,test_y)

0.8537959995579623

In [19]:
predict=logisticmodel.predict(test_x)

In [29]:
def report(y_test,predict_y):
  from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,auc
  print(f"Confusion_Matrix:{confusion_matrix(y_test,predict_y)}")
  print(f"Accuracy_Score:{accuracy_score(y_test,predict_y)}")
  print('*'*50)
  print(f"Classification_Report:{classification_report(y_test,predict_y)}")
  print('*'*50)


In [33]:
report(test_y,predict)

Confusion_Matrix:[[6239  492]
 [ 886 1432]]
Accuracy_Score:0.8477179798872804
**************************************************
Classification_Report:              precision    recall  f1-score   support

           0       0.88      0.93      0.90      6731
           1       0.74      0.62      0.68      2318

    accuracy                           0.85      9049
   macro avg       0.81      0.77      0.79      9049
weighted avg       0.84      0.85      0.84      9049

**************************************************


# using Decision tree

In [34]:
 from sklearn.tree import DecisionTreeClassifier
 decisiontreemodel=DecisionTreeClassifier()

In [35]:
para={
     'criterion':['gini', 'entropy'],
     'splitter':['best', 'random'],
     'max_depth':range(2,50),
     'max_features':['auto', 'sqrt', 'log2'],
}

In [53]:
griddecisiontree=GridSearchCV(decisiontreemodel,param_grid=para)

In [54]:
griddecisiontree.fit(train_x,train_y)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 50),
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']})

In [55]:
griddecisiontree.best_params_

{'criterion': 'entropy',
 'max_depth': 15,
 'max_features': 'auto',
 'splitter': 'best'}

In [37]:
decisiontreemodelclassfier=DecisionTreeClassifier(criterion= 'entropy',max_depth= 15,max_features='auto',splitter= 'best')

In [38]:
decisiontreemodelclassfier.fit(train_x,train_y)

DecisionTreeClassifier(criterion='entropy', max_depth=15, max_features='auto')

In [39]:
decisiontreemodelclassfier.score(test_x,test_y)

0.8319151287435076

In [40]:
predict=decisiontreemodelclassfier.predict(test_x)

In [41]:
report(test_y,predict)

Confusion_Matrix:[[6462  269]
 [1252 1066]]
Accuracy_Score:0.8319151287435076
**************************************************
Classification_Report:              precision    recall  f1-score   support

           0       0.84      0.96      0.89      6731
           1       0.80      0.46      0.58      2318

    accuracy                           0.83      9049
   macro avg       0.82      0.71      0.74      9049
weighted avg       0.83      0.83      0.82      9049

**************************************************


# random forest

In [42]:
from sklearn.ensemble import RandomForestClassifier
randomforest=RandomForestClassifier()


In [43]:
para={
     'criterion':['gini', 'entropy'],
       }

In [44]:
gridrandomforest=GridSearchCV(randomforest,param_grid=para)

In [23]:
gridrandomforest.fit(train_x,train_y)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy']})

In [24]:
gridrandomforest.best_params_


{'criterion': 'gini'}

In [45]:
randomforestmodel=RandomForestClassifier(criterion= 'gini',random_state=33)

In [46]:
randomforestmodel.fit(train_x,train_y)

RandomForestClassifier(random_state=33)

In [47]:
randomforestmodel.score(test_x,test_y)

0.8477179798872804

In [48]:
predict=randomforestmodel.predict(test_x)


In [50]:
report(test_y,predict)

Confusion_Matrix:[[6239  492]
 [ 886 1432]]
Accuracy_Score:0.8477179798872804
**************************************************
Classification_Report:              precision    recall  f1-score   support

           0       0.88      0.93      0.90      6731
           1       0.74      0.62      0.68      2318

    accuracy                           0.85      9049
   macro avg       0.81      0.77      0.79      9049
weighted avg       0.84      0.85      0.84      9049

**************************************************


# XGB CLASSIFIER

In [52]:
from xgboost import XGBClassifier
xgb=XGBClassifier(objective='binary:logistic')
param_grid={
   
    ' learning_rate':[0.1,0.01,0.001],
    'max_depth': [3,5,10,20],
    'n_estimators':[10,50,100,200]
    
}


In [53]:
grid_xgb=GridSearchCV(xgb,param_grid=param_grid)

In [54]:
grid_xgb.fit(train_x,train_y)

oss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:50:01] WARNING: ../src/learner.cc:541: 
Parameters: {  learning_rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[20:50:01] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:50:04] WARNING: ../src/learner.cc:541: 
Parameters: {  learning_rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[20:

GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, validate_parameter

In [55]:
grid_xgb.best_params_

{' learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}

In [66]:
xgbmodel=XGBClassifier(objective='binary:logistic',learning_rate= 0.1, max_depth=3, n_estimators= 100,random_state=33)

In [67]:
xgbmodel.fit(train_x,train_y)

/home/saibeast/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[21:03:30] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=33,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [68]:
xgb.score(test_x,test_y)

0.8680517184219251

In [69]:
predictxgb=xgbmodel.predict(test_x)

In [71]:
report(test_y,predictxgb)

Confusion_Matrix:[[6418  313]
 [ 922 1396]]
Accuracy_Score:0.8635208310310531
**************************************************
Classification_Report:              precision    recall  f1-score   support

           0       0.87      0.95      0.91      6731
           1       0.82      0.60      0.69      2318

    accuracy                           0.86      9049
   macro avg       0.85      0.78      0.80      9049
weighted avg       0.86      0.86      0.86      9049

**************************************************


# knn classfication

In [18]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()

In [19]:
para={
     'n_neighbors':[5,10,15,20,25],
     'weights':['uniform', 'distance'],
     'leaf_size':[25,30,35,40,45]
      }

In [20]:
gridknn=GridSearchCV(knn,param_grid=para,n_jobs=-1)

In [21]:
gridknn.fit(train_x,train_y)

GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'leaf_size': [25, 30, 35, 40, 45],
                         'n_neighbors': [5, 10, 15, 20, 25],
                         'weights': ['uniform', 'distance']})

In [22]:
gridknn.best_params_

{'leaf_size': 25, 'n_neighbors': 25, 'weights': 'distance'}

In [24]:
knnmodel=KNeighborsClassifier(leaf_size= 25, n_neighbors= 25, weights='distance')

In [25]:
knnmodel.fit(train_x,train_y)

KNeighborsClassifier(leaf_size=25, n_neighbors=25, weights='distance')

In [26]:
knnmodel.score(test_x,test_y)

0.8231848823074372

In [27]:
predictknn=knnmodel.predict(test_x)

In [30]:
report(test_y,predictknn)

Confusion_Matrix:[[6156  575]
 [1025 1293]]
Accuracy_Score:0.8231848823074372
**************************************************
Classification_Report:              precision    recall  f1-score   support

           0       0.86      0.91      0.88      6731
           1       0.69      0.56      0.62      2318

    accuracy                           0.82      9049
   macro avg       0.77      0.74      0.75      9049
weighted avg       0.81      0.82      0.82      9049

**************************************************


In [39]:
import pandas as pd
result=pd.DataFrame({'Models':['logistic_regression','DecisionTree','RandomForest','XGBclassfier','Knnclassfier'],'Model_scores':[0.854,0.832,0.847,0.868,0.823],'Model_Accuracy':[0.848,0.832,0.847,0.863,823]})
result

,Models,Model_scores,Model_Accuracy
0,logistic_regression,0.854,0.848
1,DecisionTree,0.832,0.832
2,RandomForest,0.847,0.847
3,XGBclassfier,0.868,0.863
4,Knnclassfier,0.823,823.000


# As we can come to a conclusion that XGBClassifier performes well when compared to other classfication algorithms ,by looking at the above table